# Build a basic text search application from python with Vespa

> Introducing pyvespa simplified API. Build Vespa application from python with few lines of code.

- toc: true 
- badges: false
- comments: true
- categories: [vespa, pyvespa, cord19]

This post will introduce you to the simplified `pyvespa` API that allows us to build a basic text search application from scratch with just a few code lines from python. Follow-up posts will add layers of complexity by incrementally building on top of the basic app described here.

`pyvespa` exposes a subset of [Vespa](https://vespa.ai/) API in python. The library’s primary goal is to allow for faster prototyping and facilitate Machine Learning experiments for Vespa applications. I have written about how we can use it to [connect and interact with running Vespa applications](https://towardsdatascience.com/how-to-connect-and-interact-with-search-applications-from-python-520118139f69) and [evaluate Vespa ranking functions from python](https://towardsdatascience.com/how-to-evaluate-vespa-ranking-functions-from-python-7749650f6e1a). This time, we focus on building and deploying applications from scratch.

## Install

The pyvespa simplified API introduced here was released on version `0.2.0`

`pip3 install pyvespa>=0.2.0`

## Define the application

As an example, we are going to build an application to search through [CORD19 sample data](https://ir.nist.gov/covidSubmit/data.html).

### Create an application package

The first step is to create a Vespa [ApplicationPackage](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.ApplicationPackage):

In [1]:
from vespa.package import ApplicationPackage

app_package = ApplicationPackage(name="cord19")

### Add fields to the Schema

We can then add [fields](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.Field) to the application's [Schema](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.Schema) created by default in `app_package`.

In [2]:
from vespa.package import Field

app_package.schema.add_fields(
    Field(name = "cord_uid", type = "string", indexing = ["attribute", "summary"]),
    Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
    Field(name = "abstract", type = "string", indexing = ["index", "summary"], index = "enable-bm25")
)

* `cord_uid` will store the cord19 document ids, while `title` and `abstract` are self explanatory. 

* All the fields in this case are of type `string`. 

* Including `"index"` in the `indexing` list means that Vespa will create a searchable index for `title` and `abstract`. You can read more about which options is available for `indexing` in the [Vespa documentation](https://docs.vespa.ai/documentation/reference/schema-reference.html#indexing). 

* Setting `index = "enable-bm25"` makes Vespa pre-compute quantities to make it fast to compute the bm25 score. We will use BM25 to rank the documents retrieved.

### Search multiple fields when querying

A [Fieldset](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.FieldSet) groups fields together for searching. For example, the `default` fieldset defined below groups `title` and `abstract` together.

In [3]:
from vespa.package import FieldSet

app_package.schema.add_field_set(
    FieldSet(name = "default", fields = ["title", "abstract"])
)

### Define how to rank the documents matched

We can specify how to rank the matched documents by defining a [RankProfile](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.RankProfile). In this case, we defined the `bm25` rank profile that combines that BM25 scores computed over the `title` and `abstract` fields. 

In [4]:
from vespa.package import RankProfile

app_package.schema.add_rank_profile(
    RankProfile(name = "bm25", first_phase = "bm25(title) + bm25(abstract)")
)

## Deploy your application

We have now defined a basic text search app containing relevant fields, a fieldset to group fields together and a rank profile to rank matched documents. It is time to deploy our application. We can locally deploy our `app_package` using Docker without leaving the notebook, by creating an instance of [VespaDocker](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.package.VespaDocker), as shown below:

In [5]:
from vespa.package import VespaDocker

vespa_docker = VespaDocker(port=8080)

app = vespa_docker.deploy(
    application_package = app_package,
    disk_folder="/Users/username/cord19_app" # specify your desired absolute path here
)

Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for application status.


`app` now holds a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance, which we are going to use to interact with our application. Congratulations, you now have a Vespa application up and running.

It is important to know that `pyvespa` simply provides a convenient API to define Vespa application packages from python. `vespa_docker.deploy` export Vespa configuration files to the `disk_folder` defined above. Going through those files is a nice way to start learning about Vespa syntax.

## Feed some data

Our first action after deploying a Vespa application is usually to feed some data to it.

In [6]:
from pandas import read_csv

parsed_feed = read_csv("/Users/tmartins/projects/sw/blog/_notebooks/data/2021-01-18-cord19-deploy-bert-from-pyvespa/parsed_feed.csv")
parsed_feed = parsed_feed.head(100)

In [7]:
parsed_feed

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...
...,...,...,...
95,63bos83o,Global Surveillance of Emerging Influenza Viru...,BACKGROUND: Effective influenza surveillance r...
96,hqc7u9w3,Transmission Parameters of the 2001 Foot and M...,"Despite intensive ongoing research, key aspect..."
97,87zt7lew,Efficient replication of pneumonia virus of mi...,Pneumonia virus of mice (PVM; family Paramyxov...
98,wgxt36jv,Designing and conducting tabletop exercises to...,"BACKGROUND: Since 2001, state and local health..."


In [ ]:
parsed_feed.to_

In [ ]:
for idx, row in parsed_feed.iterrows():
    fields = {
        "cord_uid": str(row["cord_uid"]),
        "title": str(row["title"]),
    }
    fields.update(
        bert_config.doc_fields(text = str(row["title"]))
    )
    response = app.feed_data_point(
        schema = "cord19",
        data_id = str(row["cord_uid"]),
        fields = fields,
    )

In [ ]:
response.json()

## Query your application

In [ ]:
from vespa.query import QueryModel, RankProfile as Ranking, OR, QueryModelFeature

result = app.query(
    query="this is a test", 
    query_model=QueryModel(
        query_properties=[
            QueryModelFeature(bert_config)
        ],
        match_phase = OR(),
        rank_profile = Ranking(name="pretrained_bert_tiny")
    )
)

In [ ]:
result.json

In [ ]:
result.number_documents_retrieved